# Relativemomentum strategy

In [89]:
import pandas as pd
import numpy as np
import datetime
import FinanceDataReader as fdr
import matplotlib.pyplot as plt
plt.style.use(['fivethirtyeight'])


In [90]:
def data_processing(sample, ticker, base_date):
    sample.reset_index(inplace=True)
    sample.reset_index(inplace=True, drop=True)
    sample['CODE'] = ticker
    sample = sample[sample['Date'] >= base_date][['Date', 'CODE', 'Close']].copy()
    sample.reset_index(inplace=True, drop=True)
    sample['STD_YM'] = sample['Date'].map(lambda x : pd.Timestamp(x + pd.offsets.MonthEnd(n=0)).strftime('%Y-%m'))
    sample['1M_RET'] = 0.0
    ym_keys = list(sample['STD_YM'].unique())
    return sample, ym_keys

In [95]:
month_last_df = pd.DataFrame(columns=['Date', 'CODE', 'Close', '1M_RET'])
stock_df = pd.DataFrame(columns=['Date', 'CODE', 'Close'])
stocks = pd.read_csv('./stock_codes.csv')
stocks['code'] = stocks['code'].map(lambda x : x[1:])
stocks.set_index('code', inplace=True)
print(stocks)

              label
code               
005930  SAMSUNGelec
051910       LGchem
000660      SKhynix
207940   SAMSUNGbio
035420        NAVER
035720        KAKAO
005380    HYNDAIcar
006400   SAMSUNGsdi
068270    CELLTRION
000270          KIA


In [96]:

dataframes = []

for st in stocks.index:
    read_df = fdr.DataReader(st)
    price_df, ym_keys = data_processing(read_df, stocks.loc[st,'label'], base_date='2018-01-02')
    stock_df = stock_df.append(price_df.loc[:, ['Date', 'CODE', 'Close']], sort=False)
    for ym in ym_keys:
        m_ret = price_df.loc[price_df[price_df['STD_YM'] == ym].index[-1], 'Close']\
              / price_df.loc[price_df[price_df['STD_YM'] == ym].index[0], 'Close']
        price_df.loc[price_df['STD_YM'] == ym, ['1M_RET']] = m_ret
        month_last_df = month_last_df.append(price_df.loc[price_df[price_df['STD_YM'] == ym].index[-1], ['Date', 'CODE', '1M_RET']])
      

In [97]:
month_last_df

,Date,CODE,1M_RET,Close
21,2018-01-31,SAMSUNGelec,0.978048,NaN
39,2018-02-28,SAMSUNGelec,0.944601,NaN
60,2018-03-30,SAMSUNGelec,1.069535,NaN
81,2018-04-30,SAMSUNGelec,1.091883,NaN
101,2018-05-31,SAMSUNGelec,0.956604,NaN
...,...,...,...,...
838,2021-05-31,KIA,1.068664,NaN
860,2021-06-30,KIA,1.045508,NaN
882,2021-07-30,KIA,0.923757,NaN
903,2021-08-31,KIA,1.004723,NaN


In [70]:
month_ret_df = month_last_df.pivot('Date', 'CODE', '1M_RET').copy()
month_ret_df

CODE,CELLTRION,HYNDAIcar,KAKAO,KIA,LGchem,NAVER,SAMSUNGbio,SAMSUNGelec,SAMSUNGsdi,SKhynix
Date,,,,,,,,,,
2018-01-31,1.397522,1.083612,0.955620,1.056402,1.051095,1.028248,1.125802,0.978048,0.929245,0.959530
2018-02-28,1.177456,1.006231,0.904581,1.000000,0.915173,0.869990,1.014656,0.944601,0.912000,1.032258
2018-03-30,0.831989,0.919872,1.035286,0.925373,1.005215,1.007635,1.046187,1.069535,1.129032,1.050388
2018-04-30,0.891446,1.077441,0.860459,1.060897,0.929032,0.912099,1.004115,1.091883,0.919799,1.039360
2018-05-31,1.025093,0.868750,0.903925,0.942511,0.998525,0.906508,1.076733,0.956604,1.095628,1.123947
2018-06-29,1.109693,0.896429,1.095738,0.967085,0.989614,1.122055,0.923588,0.909357,1.023923,0.937637
2018-07-31,0.894560,1.048583,1.008982,1.029221,1.181102,0.953395,0.899879,1.015368,1.070093,1.031063
2018-08-31,0.962570,0.972763,1.111116,1.009449,0.977303,1.032968,1.208877,1.040816,1.044248,0.969626
2018-09-28,1.112360,1.036000,0.979415,1.103774,0.993207,0.949601,1.131356,0.978925,1.065979,0.906948


In [71]:
month_ret_df = month_ret_df.rank(axis=1, ascending=False, method='max', pct=True)
month_ret_df

CODE,CELLTRION,HYNDAIcar,KAKAO,KIA,LGchem,NAVER,SAMSUNGbio,SAMSUNGelec,SAMSUNGsdi,SKhynix
Date,,,,,,,,,,
2018-01-31,0.1,0.3,0.9,0.4,0.5,0.6,0.2,0.7,1.0,0.8
2018-02-28,0.1,0.4,0.9,0.5,0.7,1.0,0.3,0.6,0.8,0.2
2018-03-30,1.0,0.9,0.5,0.8,0.7,0.6,0.4,0.2,0.1,0.3
2018-04-30,0.9,0.2,1.0,0.3,0.6,0.8,0.5,0.1,0.7,0.4
2018-05-31,0.4,1.0,0.9,0.7,0.5,0.8,0.3,0.6,0.2,0.1
2018-06-29,0.2,1.0,0.3,0.6,0.5,0.1,0.8,0.9,0.4,0.7
2018-07-31,1.0,0.3,0.7,0.5,0.1,0.8,0.9,0.6,0.2,0.4
2018-08-31,1.0,0.8,0.2,0.6,0.7,0.5,0.1,0.4,0.3,0.9
2018-09-28,0.2,0.5,0.7,0.3,0.6,0.9,0.1,0.8,0.4,1.0


In [72]:
month_ret_df = month_ret_df.where(month_ret_df < 0.4, np.nan)
month_ret_df.fillna(0, inplace=True)
month_ret_df[month_ret_df != 0] = 1
stock_codes = list(stock_df['CODE'].unique())
month_ret_df.tail(10)

CODE,CELLTRION,HYNDAIcar,KAKAO,KIA,LGchem,NAVER,SAMSUNGbio,SAMSUNGelec,SAMSUNGsdi,SKhynix
Date,,,,,,,,,,
2020-12-30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
2021-01-29,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
2021-02-26,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
2021-03-31,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2021-04-30,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
2021-05-31,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2021-06-30,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2021-07-30,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
2021-08-31,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [73]:
def create_trade_book(sample, sample_codes):
    book = pd.DataFrame()
    book = sample[sample_codes].copy()
    book['STD_YM'] = sample.index.map(lambda x : pd.Timestamp(x + pd.offsets.MonthEnd(n=0)).strftime('%Y-%m'))
    for c in sample_codes:
        book['p ' + c] = ''
        book['r ' + c] = ''
    return book

In [74]:
def tradings(book, s_codes):
    std_ym = ''
    buy_phase = False
    
    for s in s_codes:
        print(s)
        for i in book.index:
            if book.loc[i,'p '+ s] == '' and book.shift(1).loc[i,'p '+ s] == 'ready ' + s:
                std_ym = book.loc[i, 'STD_YM']
                buy_phase = True
            
            if book.loc[i,'p '+ s] == ''and book.loc[i, 'STD_YM'] == std_ym and buy_phase == True:
                book.loc[i,'p '+ s] = 'buy ' + s
            
            if book.loc[i,'p '+ s] == '':
                std_ym = None
                buy_phase = False
    return book

In [75]:
def multi_returns(book, s_codes):
    rtn = 1.0
    buy_dict = {}
    num = len(s_codes)
    sell_dict = {}
    
    for i in book.index:
        for s in s_codes:
            if book.loc[i,'p '+ s] == 'buy '+ s and \
            book.shift(1).loc[i,'p '+ s] == 'ready '+ s and \
            book.shift(2).loc[i,'p '+ s] == '':
                buy_dict[s] = book.loc[i, s]
            elif book.loc[i,'p '+ s] == '' and book.shift(1).loc[i,'p '+ s] == 'buy '+ s:
                sell_dict[s] = book.loc[i, s]
                rtn = (sell_dict[s] / buy_dict[s]) - 1
                book.loc[i, 'r '+ s] = rtn
                print('개별 청산일 : ', i, '종목 코드 : ', s, 'long 진입가격 : ', buy_dict[s], \
                      ' | long 청산가격 : ', sell_dict[s], ' | return:', round(rtn * 100, 2), '%')
                        
            if book.loc[i,'p '+ s] == '':
                buy_dict[s] = 0.0
                sell_dict[s] = 0.0
                
    acc_rtn = 1.0
    
    for i in book.index:
        rtn = 0.0
        count = 0
        for s in s_codes:
            if book.loc[i,'p '+ s] == '' and book.shift(1).loc[i,'p '+ s] == 'buy '+ s:
                count += 1
                rtn += book.loc[i, 'r '+ s]
        
        if (rtn != 0.0) & (count != 0) :
            acc_rtn *= (rtn/count) + 1
            print('누적 청산일 : ', i, '청산 종목수 : ', count, \
                  '청산 수익률 : ', round((rtn/count), 4), '누적 수익률 : ', round(acc_rtn, 4))
            
        book.loc[i,'acc_rtn'] = acc_rtn
        
    print('누적 수익률 :', round(acc_rtn, 4))
    
    

In [76]:
sig_dict = dict()
for date in month_ret_df.index:
    ticker_list = list(month_ret_df.loc[date, month_ret_df.loc[date,:] >= 1.0].index)
    sig_dict[date] = ticker_list
stock_c_matrix = stock_df.pivot('Date', 'CODE', 'Close').copy()
book = create_trade_book(stock_c_matrix, list(stock_df['CODE'].unique()))

for date, values in sig_dict.items():
    for stock in values:
        book.loc[date, 'p '+ stock] = 'ready ' + stock

book = tradings(book, stock_codes)

book

SAMSUNGelec
LGchem
SKhynix
SAMSUNGbio
NAVER
KAKAO
HYNDAIcar
SAMSUNGsdi
CELLTRION
KIA


CODE,SAMSUNGelec,LGchem,SKhynix,SAMSUNGbio,NAVER,KAKAO,HYNDAIcar,SAMSUNGsdi,CELLTRION,KIA,...,p KAKAO,r KAKAO,p HYNDAIcar,r HYNDAIcar,p SAMSUNGsdi,r SAMSUNGsdi,p CELLTRION,r CELLTRION,p KIA,r KIA
Date,,,,,,,,,,,,,,,,,,,,,
2018-01-02,51020,411000,76600,389500,177251,29405,149500,212000,208167,32800,...,,,,,,,,,,
2018-01-03,51620,409500,77700,391000,174447,29906,150500,207500,227334,32600,...,,,,,,,,,,
2018-01-04,51080,395500,77100,383000,178853,31311,146500,208500,230376,31550,...,,,,,,,,,,
2018-01-05,52120,405000,79300,390000,181857,31311,149000,220500,245949,31950,...,,,,,,,,,,
2018-01-08,52020,424500,78200,399500,190269,32014,151000,225500,278754,32400,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-09-10,75300,740000,105000,925000,410000,130000,206500,750000,267500,82500,...,buy KAKAO,,,,buy SAMSUNGsdi,,buy CELLTRION,,,
2021-09-13,76300,747000,106500,913000,408000,124500,205000,741000,262000,82600,...,buy KAKAO,,,,buy SAMSUNGsdi,,buy CELLTRION,,,
2021-09-14,76600,734000,107500,913000,402500,124000,209000,755000,263000,83400,...,buy KAKAO,,,,buy SAMSUNGsdi,,buy CELLTRION,,,


In [77]:
multi_returns(book, stock_codes)

개별 청산일 :  2018-03-02 00:00:00 종목 코드 :  HYNDAIcar long 진입가격 :  160500  | long 청산가격 :  156000  | return: -2.8 %
개별 청산일 :  2018-04-02 00:00:00 종목 코드 :  SAMSUNGbio long 진입가격 :  443500  | long 청산가격 :  486000  | return: 9.58 %
개별 청산일 :  2018-04-02 00:00:00 종목 코드 :  CELLTRION long 진입가격 :  276266  | long 청산가격 :  280136  | return: 1.4 %
개별 청산일 :  2018-05-02 00:00:00 종목 코드 :  SKhynix long 진입가격 :  77400  | long 청산가격 :  83100  | return: 7.36 %
개별 청산일 :  2018-05-02 00:00:00 종목 코드 :  SAMSUNGsdi long 진입가격 :  199500  | long 청산가격 :  183000  | return: -8.27 %
개별 청산일 :  2018-06-01 00:00:00 종목 코드 :  SAMSUNGelec long 진입가격 :  48540  | long 청산가격 :  51300  | return: 5.69 %
개별 청산일 :  2018-06-01 00:00:00 종목 코드 :  HYNDAIcar long 진입가격 :  160000  | long 청산가격 :  140000  | return: -12.5 %
개별 청산일 :  2018-06-01 00:00:00 종목 코드 :  KIA long 진입가격 :  33050  | long 청산가격 :  31900  | return: -3.48 %
개별 청산일 :  2018-07-02 00:00:00 종목 코드 :  SKhynix long 진입가격 :  91400  | long 청산가격 :  83700  | return: -8.42 %
개별 청산일 :  2018-07-02 

개별 청산일 :  2020-12-01 00:00:00 종목 코드 :  SAMSUNGbio long 진입가격 :  698000  | long 청산가격 :  793000  | return: 13.61 %
개별 청산일 :  2020-12-01 00:00:00 종목 코드 :  SAMSUNGsdi long 진입가격 :  440000  | long 청산가격 :  547000  | return: 24.32 %
개별 청산일 :  2020-12-01 00:00:00 종목 코드 :  KIA long 진입가격 :  50500  | long 청산가격 :  59800  | return: 18.42 %
개별 청산일 :  2021-01-04 00:00:00 종목 코드 :  LGchem long 진입가격 :  809000  | long 청산가격 :  889000  | return: 9.89 %
개별 청산일 :  2021-01-04 00:00:00 종목 코드 :  CELLTRION long 진입가격 :  337811  | long 청산가격 :  347500  | return: 2.87 %
개별 청산일 :  2021-02-01 00:00:00 종목 코드 :  SAMSUNGelec long 진입가격 :  83000  | long 청산가격 :  83000  | return: 0.0 %
개별 청산일 :  2021-02-01 00:00:00 종목 코드 :  SKhynix long 진입가격 :  100500  | long 청산가격 :  125000  | return: 24.38 %
개별 청산일 :  2021-02-01 00:00:00 종목 코드 :  SAMSUNGsdi long 진입가격 :  671000  | long 청산가격 :  745000  | return: 11.03 %
개별 청산일 :  2021-03-02 00:00:00 종목 코드 :  KIA long 진입가격 :  90000  | long 청산가격 :  80100  | return: -11.0 %
개별 청산일 :  2021-04-01 00

In [78]:
print(month_ret_df.loc[book.index[-1].strftime('%Y-%m-%d')])


CODE
CELLTRION      0.0
HYNDAIcar      0.0
KAKAO          0.0
KIA            1.0
LGchem         1.0
NAVER          0.0
SAMSUNGbio     0.0
SAMSUNGelec    1.0
SAMSUNGsdi     0.0
SKhynix        0.0
Name: 2021-09-16 00:00:00, dtype: float64


In [79]:
stock_df

,Date,CODE,Close
0,2018-01-02,SAMSUNGelec,51020
1,2018-01-03,SAMSUNGelec,51620
2,2018-01-04,SAMSUNGelec,51080
3,2018-01-05,SAMSUNGelec,52120
4,2018-01-08,SAMSUNGelec,52020
...,...,...,...
911,2021-09-10,KIA,82500
912,2021-09-13,KIA,82600
913,2021-09-14,KIA,83400
914,2021-09-15,KIA,83300


In [80]:
month_last_df

,Date,CODE,1M_RET
21,2018-01-31,SAMSUNGelec,0.978048
39,2018-02-28,SAMSUNGelec,0.944601
60,2018-03-30,SAMSUNGelec,1.069535
81,2018-04-30,SAMSUNGelec,1.091883
101,2018-05-31,SAMSUNGelec,0.956604
...,...,...,...
838,2021-05-31,KIA,1.068664
860,2021-06-30,KIA,1.045508
882,2021-07-30,KIA,0.923757
903,2021-08-31,KIA,1.004723
